# Glacier Gallery

In [ ]:
# https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds

from bokeh.models import Legend, LegendItem
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS

from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.plotting import figure


from bokeh.models.markers import Circle

hv.extension('bokeh', width=100)

In [ ]:
# prepare coordinates
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

# build source (Columndatasource)
def build_source(dataframe, glacier_type):
    """
    glacier_type: str
    """
    source = ColumnDataSource(
                data=dict(
                        x=list(dataframe.loc[dataframe['type']            == glacier_type, 'coords_x']), 
                        y=list(dataframe.loc[dataframe['type']            == glacier_type, 'coords_y']),
                        lon=list(dataframe.loc[dataframe['type']          == glacier_type, 'cenlon']), 
                        lat=list(dataframe.loc[dataframe['type']          == glacier_type, 'cenlat']),
                        Area=list(dataframe.loc[dataframe['type']         == glacier_type, 'area_km2']),
                        img=list(dataframe.loc[dataframe['type']          == glacier_type, 'images']),
                        gltype=list(dataframe.loc[dataframe['type']       == glacier_type, 'type']),
                        gltype_text=list(dataframe.loc[dataframe['type']  == glacier_type, 'glaciertype_text']),
                        empty1=['']*len(dataframe.loc[dataframe['type']   == glacier_type]),
                        empty2=['']*len(dataframe.loc[dataframe['type']   == glacier_type]),
                        name=list(dataframe.loc[dataframe['type']         == glacier_type, 'name']),
                        RGI_id=list(dataframe.loc[dataframe['type']       == glacier_type, 'rgi_id'])
                        #prcp=list(df['avg_prcp']),
                        #gtype=list(df['glacier_type']),)
                       ))
    return source

In [ ]:
# load data
df_all = pd.read_csv('https://raw.githubusercontent.com/pyviz-demos/glaciers/master/data/oggm_glacier_explorer.csv')

In [ ]:
# which one is columbia? : RGI60-01.10689 or RGI60-02.18415 
glacier_list = ['RGI60-01.10299', 'RGI60-01.10689', 'RGI60-01.16195', 'RGI60-03.00840', 'RGI60-03.04539', 'RGI60-08.01097', 'RGI60-11.00746', 'RGI60-11.00787', 'RGI60-11.00897', 'RGI60-11.01270', 'RGI60-11.01328', 'RGI60-11.01450', 'RGI60-11.02709', 'RGI60-11.03638', 'RGI60-11.03643', 'RGI60-13.04946', 'RGI60-16.02207', 'RGI60-16.02444', 'RGI60-18.02342']
df = df_all.loc[df_all['rgi_id'].isin(glacier_list)]

glacier_frame =  df.rgi_id.to_frame()
glacier_frame['name'] = ['Coxe Glacier', 'Columbia', 'South Glacier', ' Baby Glacier', 'White Glacier', 'Briksdalsbreen', 'Gepatschferner', 'Kesselwandferner', 'Hintereisferner', 'Oberer Grindelwald Glacier', 'Unteraargletscher', 'Großer Aletschgletscher', 'Mont Mine Glacier', 'Argentière', 'Mer de Glace', 'Samoilowich', 'Shallap Glacier', 'Artesonraju', 'Tasman Glacier']
df.loc[:, 'name'] = glacier_frame['name']
#suldenfern = ['', 46.497838, 10.564395, '', '', '', '', '', '', '', '', 'SULDENFERNER']
#suldenfern = suldenfern.to_frame()
#df.append(suldenfern)
df['Coords'] = list(zip(round(df.cenlat,2), round(df.cenlon,2)))
#df.tail()

In [ ]:
# prepare "background glacier" in the map
#df_short = df_all[::2]
df_short = df_all

df_short['Coords'] = list(zip(round(df_short.cenlat,2), round(df_short.cenlon,2)))
df_short.loc[:, 'coords_x'] = df_short['Coords'].apply(lambda x: merc(x)[0])
df_short.loc[:, 'coords_y'] = df_short['Coords'].apply(lambda x: merc(x)[1])

In [ ]:
# add images
url = 'https://upload.wikimedia.org/wikipedia/commons/9/95/SantaCruz-PeritoMoreno-P2140146b.jpg'
    # more glacier example pictures
#           'https://www.goodfreephotos.com/albums/argentina/other-argentina/perito-moreno-glacier-argentina.jpg',
#           'https://upload.wikimedia.org/wikipedia/commons/1/18/South_Sawyer_Glacier_48.jpg',
#           'https://upload.wikimedia.org/wikipedia/commons/8/8b/Picture_of_K2.jpg'],

df.loc[:,'images'] = url
df.loc[df['name']== 'Hintereisferner', 'images'] = 'https://www.swisseduc.ch/glaciers/alps/hintereisferner/icons/hintereisferner_overview.jpg'


#df.loc[df['glacier_name']== 'SULDENFERNER', 'images'] = 'https://upload.wikimedia.org/wikipedia/commons/8/84/Schaubachh%C3%BCtte_Panorama.jpg'

In [ ]:
# add glacier type 
df.loc[:, 'type'] = 'Pure ice glacier'
df.loc[df['name']== 'Coxe Glacier', 'type'] = 'Calving glacier'
df.loc[df['name']== 'Columbia', 'type'] = 'Calving glacier'

# df.loc[df['glacier_name']== 'SULDENFERNER', 'type'] = 'Debris covered glacier'

# add glacier type description
calving_glacier_text = 'Land terminating or water terminating. Loose calves'
rock_glacier_text = 'These glaciers contain at least 50% rocks in the ice...'
ice_glacier_text = ' Beautiful white blue shimmering glaciers...'
df.loc[df['type'] == 'Pure ice glacier', 'glaciertype_text'] = ice_glacier_text
df.loc[df['type'] == 'Calving glacier', 'glaciertype_text'] = calving_glacier_text
df.loc[df['type'] == 'Rock glacier', 'glaciertype_text'] = rock_glacier_text
df.loc[df['type'] == 'Pure ice glacier', 'color'] = 'blue'
df.loc[df['type'] == 'Calving glacier', 'color'] = 'red'
df.loc[df['type'] == 'Rock glacier', 'color'] = 'brown'


In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])
############################################################################
# build sources
source_calving = build_source(df, 'Calving glacier')
source_debris = build_source(df, 'Debris covered glacier')
source_pure = build_source(df, 'Pure ice glacier')

source = ColumnDataSource(
    data=dict(
                        x=list(df['coords_x']), 
                        y=list(df['coords_y']),
                        Area=list(df['area_km2']),
                        #prcp=list(df['avg_prcp']),
                        #gtype=list(df['glacier_type']),
                        img=list(df['images']),
                        gltype=list(df['type']),
                        gltype_text=list(df['glaciertype_text']),
                        empty1=[''] * len(df),
                        empty2=[''] * len(df),
                        name=list(df['name']),
                        colors=list(df['color']),
                        RGI_id=list(df['rgi_id']))
                       )

df_short = ColumnDataSource(
    data=dict(
                        x=list(df_short['coords_x']), 
                        y=list(df_short['coords_y']),
                        empty1=[''] * len(df_short),
                        empty2=[''] * len(df_short),
                        RGI_id=list(df_short['rgi_id']))
                       )

In [ ]:
############################################################################
# Hover
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@img" height="150" alt="Bild" width="200"
                style="float: left; margin: 0px 50px 50px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@name</span>
        </div>
        <div>
            <span style="font-size: 15px;">@gltype</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">@lat, @lon</span>
        </div>
    </div>
"""

hover = HoverTool()#(names=['needshover1', 'needshover2', 'needshover3'])
hover.tooltips = TOOLTIPS

############################################################################
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
            plot_width = 1000,
           plot_height = 650,
            background='black',
           tools=[hover, 'tap', 'wheel_zoom', 'reset', 'pan'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

############################################################################
# add glacier dots
p.circle(x='x',
         y='y', 
         source=source,
         size=10,
         color='colors',
         #line_color="black", 
         #fill_color="black",
         fill_alpha=0.3,
         name='needshover0',
            legend='gltype'
        )


############################################################################
# build a text box with information about glacier types
def construct_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=320, min_border=0, 
        plot_width=260
        )                                                                           
    # Add the writing 
    # textstyle:    https://bokeh.pydata.org/en/latest/docs/user_guide/styling.html#fill-properties
    country1 = Text(x=10, y=100, text='empty1', text_color="#85a9c1", text_alpha=1.0, text_font='helvetica', text_font_size='10pt')# angle, angle_units, js_event_callbacks, js_property_callbacks, name, subscribed_events, 
                    #tags, text_align, text_baseline, text_color, text_font, text_font_style, text_line_height, x, x_offset, y, y_offset)
    country2 = Text(x=10, y=1000, text='empty2', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    plot.add_glyph(source, country1)
    plot.add_glyph(source, country2)    

    return plot

JS_CODE ="""
    // Get index of current selected datapoint
    sel_point_index = cb_data.source.attributes.selected["1d"]["indices"][0];     

    /* replace all name_display values with the name_label
       value of currently selected point */
    for (i=0; i < cb_data.source.data.gltype.length; i++) {
        cb_data.source.data.empty2[i] = cb_data.source.data.gltype[sel_point_index];
        }       
    cb_data.source.change.emit();
    
    for (i=0; i < cb_data.source.data.gltype.length; i++) {
        cb_data.source.data.empty1[i] = cb_data.source.data.gltype_text[sel_point_index];
        }      
     cb_data.source.change.emit();

    """

newtaptool = p.select(type=TapTool)#, names=['needshover0'])#'needshover1', 'needshover2', 'needshover3', 'needshover0'])
newtaptool.callback = CustomJS(code=JS_CODE)
text_box = construct_text_box(source)

#text_box = construct_text_box(source_calving)
#text_box = construct_text_box(source_debris)
#text_box = construct_text_box(source_pure)

############################################################################
## legend

#li4 = LegendItem(label='Glaciers',                renderers=[p.renderers[1]])
#li1 = LegendItem(label='Pure ice glaciers',       renderers=[p.renderers[1]])
#li2 = LegendItem(label='Debris covered glaciers', renderers=[p.renderers[2]])
#li3 = LegendItem(label='Calving glaciers',        renderers=[p.renderers[3]])


#legend1 = Legend(items=[li1, li2, li3], location='top_right', click_policy='hide')
#legend1 = Legend(items=[li1, li2, li3, li4], location='top_right', click_policy='hide')
#legend1 = Legend(items=[li4], location='top_right', click_policy='hide')

#p.add_layout(legend1)
p.legend.click_policy="hide"

############################################################################
# Design Application
# Panel
title       = '<div style="font-size:35px">Glacier Gallery</div>'
instruction = 'The blue areas mark glaciers. Move the mouse over the blue dots to show pictures and information. Click on it and get information about the glacier type.<br>'
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=140></a>'

header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Pane(title, width=400), pn.Pane(instruction, width=500)),
                pn.layout.HSpacer(), pn.layout.Spacer(height=20), 
                pn.Pane(pn_logo, width=140))

# Put plot and text together and show it
pn.Column(header, pn.Spacer(height=15), pn.Row(p, pn.Spacer(width=5), text_box), width_policy='max', height_policy='max').servable()

In [ ]:
p.circle(x='x',
         y='y', 
         source=source_pure,
         size=15,
         line_color="black", 
         fill_color="#85a9c1",
         fill_alpha=0.7,
         name='needshover1',
        legend='gltype'
        )
    
p.circle(x='x',
             y='y', 
             source=source_debris,
             size=15,
             line_color="black", 
             fill_color="green",
             fill_alpha=0.4,
             name='needshover2')

p.circle(x='x',
             y='y', 
             source=source_calving,
             size=15,
             line_color="black", 
             fill_color="red",
             fill_alpha=0.4,
             name='needshover3')


In [ ]:
p.circle(x='x',
         y='y', 
         source=df_short,
         size=3,
        # line_color="#85a9c1", 
         line_color="grey",
         fill_color="grey",
         fill_alpha=0.4)